This notebook calculates monthly means of O2 for model output NETCDF files. 

In [1]:
import h5py
import netCDF4
from netCDF4 import Dataset
import numpy as np
import numpy.ma as ma
import glob

In [2]:
runid='CREG025_LIM3_CANOE-VJC014l'
#variable='salt'; vunit='psu'
variable='temp'; vunit='degree C'
flist = glob.glob("/gpfs/fs7/dfo/hpcmc/comda/stod000/RUN_DIR/Auto-restart/CREG025_LIM3_CANOE/"+runid+"/CDF/"+runid+"_5d_grid_T_2017*.nc")
flist.sort()
N5=len(flist)
print(N5)

# open an arbitrary file to get basic dimension information
filein = flist[0]
data = Dataset(filein, "r", format="NETCDF4")
X = data.variables[variable][:]
print(np.shape(X))
[nz,ny,nx] = np.shape(X.squeeze())
print(nz,ny,nx)
print(np.min(X),np.max(X),np.mean(X),np.median(X))


73


/tmp/ipykernel_532622/2690282825.py:12: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = data.variables[variable][:]


(1, 75, 735, 528)
75 735 528
-2.1205952 26.929783 1.8041726272738507 0.0


/home/fid000/WORK7/software/anaconda3/envs/pypkg/lib/python3.8/site-packages/numpy/core/fromnumeric.py:758: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [3]:
# get tmask information
# tmask has 1's over ocean and zeros over land
# tmask has a t dimension but only 1 data point

filemesh='/gpfs/fs7/dfo/hpcmc/comda/stod000/CREG25/CREG025_mesh_mask.nc'
data = Dataset(filemesh, "r", format="NETCDF4")
M = data.variables['tmask'][:].squeeze()
print(np.shape(M))
print(np.min(M),np.max(M),np.mean(M),np.median(M))

# get axis data (better to use data in tmask file because model o/p files have 0 over land in nav_lon, nav_lat arrays)

xlon = data.variables['nav_lon'][:]
xlat = data.variables['nav_lat'][:]
xdpth = data.variables['nav_lev'][:]
#depth[:]=xdpth
print(np.shape(xlon))
print(np.shape(xlat))
#print(np.shape(depth))#print(np.shape(xdpth))

t12=np.array(np.arange(12)+0.5)
print(t12)
print(type(t12))


/tmp/ipykernel_532622/396071799.py:7: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  M = data.variables['tmask'][:].squeeze()
/tmp/ipykernel_532622/396071799.py:13: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  xlon = data.variables['nav_lon'][:]


(75, 735, 528)
0 1 0.3011297327011613 0.0
(735, 528)
(735, 528)
[ 0.5  1.5  2.5  3.5  4.5  5.5  6.5  7.5  8.5  9.5 10.5 11.5]
<class 'numpy.ndarray'>


/tmp/ipykernel_532622/396071799.py:14: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  xlat = data.variables['nav_lat'][:]
/tmp/ipykernel_532622/396071799.py:15: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  xdpth = data.variables['nav_lev'][:]


In [4]:
# [xb, x5, xa, xtot] 
# xb = number of days at beginning to be added from file that overlaps previous month (e.g., 4 for Feb as file starts on Jan 31)
# xa = number of days at end to be added from file that overlaps next month (e.g., 1 for Jan)
# x5 = number of full 5 day blocks to average (it's actually x5-1 if xb==0; see next cell)
# xtot = number of days in the month

timedata={
    "jan": tuple([0, 6, 1, 31]),
    "feb": tuple([4, 4, 4, 28]),
    "mar": tuple([1, 6, 0, 31]),      
    "apr": tuple([0, 6, 0, 30]),      
    "may": tuple([0, 6, 1, 31]),      
    "jun": tuple([4, 5, 1, 30]),
    "jul": tuple([4, 5, 2, 31]),
    "aug": tuple([3, 5, 3, 31]),
    "sep": tuple([2, 5, 3, 30]),
    "oct": tuple([2, 5, 4, 31]),
    "nov": tuple([1, 5, 4, 30]),
    "dec": tuple([1, 6, 0, 31])    
    }

print(timedata)


{'jan': (0, 6, 1, 31), 'feb': (4, 4, 4, 28), 'mar': (1, 6, 0, 31), 'apr': (0, 6, 0, 30), 'may': (0, 6, 1, 31), 'jun': (4, 5, 1, 30), 'jul': (4, 5, 2, 31), 'aug': (3, 5, 3, 31), 'sep': (2, 5, 3, 30), 'oct': (2, 5, 4, 31), 'nov': (1, 5, 4, 30), 'dec': (1, 6, 0, 31)}


In [5]:
# https://stackoverflow.com/questions/40554446/how-to-stop-execution-of-all-cells-in-jupyter-notebook
# from https://stackoverflow.com/questions/24005221/ipython-notebook-early-exit-from-cell?rq=3
class StopExecution(Exception):
    def _render_traceback_(self):
        return []

variable='O2'
variants = ['016f']
#variants=['i','j','k','l']#,'e','f','g']
#variants=['l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
for asuff in variants:
    runid='CREG025_LIM3_CANOE-VJC'+asuff
    #runid='CREG025_LIM3_CANOE-VJC015'+asuff
    print(runid)
    data_all=np.zeros((12,nz,ny,nx))

    flist = glob.glob("/gpfs/fs7/dfo/hpcmc/comda/stod000/RUN_DIR/Auto-restart/CREG025_LIM3_CANOE/"+runid+"/CDF/"
                      +runid+"_5d_ptrc_T_2017*.nc")
    flist.sort()
    
    ifile=0
    imon=0

    for mmon in ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']:
        xb=timedata[mmon][0]     # days before (shared with previous month)
        x5=timedata[mmon][1]     # number of 5 day blocks
        xa=timedata[mmon][2]     # days after (shared with next month)
        xtot=timedata[mmon][3]

        print(xb,x5,xa,xtot)
        if (5*x5+xb+xa)!=xtot:
            raise StopExecution("Numbers do not add up")

        X=np.zeros((nz,ny,nx))

        if (xb==0):
            filein = flist[ifile]
            print(filein)
            data = Dataset(filein, "r", format="NETCDF4")
            Y0 = data.variables[variable][:]
    # https://www.educative.io/answers/how-to-get-the-data-of-a-masked-array-as-an-ndarray
            Y1 = ma.getdata(Y0).squeeze()
            X = X+Y1*M*5.
            x5=x5-1        # decrement x5 to account for reading a file here
        else:
            Y0 = data.variables[variable][:]
            Y1 = ma.getdata(Y0).squeeze()
            X = X+Y1*M*float(xb)

        for i5 in np.arange(x5):
            ifile=ifile+1
            filein = flist[ifile]
            print(filein)
            data = Dataset(filein, "r", format="NETCDF4")
            Y0 = data.variables[variable][:]
            Y1 = ma.getdata(Y0).squeeze()
            X = X+Y1*M*5.

        ifile=ifile+1
        if (xa>0):
            filein = flist[ifile]
            print(filein)
            data = Dataset(filein, "r", format="NETCDF4")
            Y0 = data.variables[variable][:]
            Y1 = ma.getdata(Y0).squeeze()
            X = X+Y1*M*float(xa)

        X=X/float(xtot)
        print( np.min(X), np.max(X), np.mean(X), np.median(X))

        data_all[imon,:,:,:] = X
        imon=imon+1   
        
    

    mdv=1.e20
    M4 = M + data_all*0.
    data_all=data_all+(1.-M4)*mdv
    
    fileout='/home/fid000/WORK7/ANALYSIS/model_evaluation/MonthlyMeans/O2/'+runid+'_monthly_'+variable+'.nc'
    print(fileout)

    fx = Dataset(fileout, "w", format="NETCDF4")
    x = fx.createDimension("x", nx);
    y = fx.createDimension("y", ny);
    z = fx.createDimension("z", nz);
    time = fx.createDimension("time", 12);

    lat = fx.createVariable('nav_lat', np.float32, ('y','x'))
    lat.units = 'degrees_north'
    lat.long_name = 'latitude'

    lon = fx.createVariable('nav_lon', np.float32, ('y','x'))
    lon.units = 'degrees_east'
    lon.long_name = 'longitude'

    depth = fx.createVariable('level', np.float32, ('z',))
    depth.units = 'm'
    depth.long_name = 'level'

    time = fx.createVariable('time', np.float32, ('time',))
    time.units = 'months'
    time.long_name = 'time'

    data = fx.createVariable(variable, np.float32, ('time','z','y','x'), fill_value=mdv)
    data.units = vunit
    data.long_name = variable
    #data._FillValue = mdv

    lon[:,:]=xlon; lat[:,:]=xlat; depth[:]=xdpth; time[:]=t12
    data[:,:,:,:]=data_all

    fx.close()

CREG025_LIM3_CANOE-VJC016f
0 6 1 31
/gpfs/fs7/dfo/hpcmc/comda/stod000/RUN_DIR/Auto-restart/CREG025_LIM3_CANOE/CREG025_LIM3_CANOE-VJC016f/CDF/CREG025_LIM3_CANOE-VJC016f_5d_ptrc_T_20170101-20170105.nc


/tmp/ipykernel_532622/3677473921.py:40: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Y0 = data.variables[variable][:]


/gpfs/fs7/dfo/hpcmc/comda/stod000/RUN_DIR/Auto-restart/CREG025_LIM3_CANOE/CREG025_LIM3_CANOE-VJC016f/CDF/CREG025_LIM3_CANOE-VJC016f_5d_ptrc_T_20170106-20170110.nc


/tmp/ipykernel_532622/3677473921.py:55: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Y0 = data.variables[variable][:]


/gpfs/fs7/dfo/hpcmc/comda/stod000/RUN_DIR/Auto-restart/CREG025_LIM3_CANOE/CREG025_LIM3_CANOE-VJC016f/CDF/CREG025_LIM3_CANOE-VJC016f_5d_ptrc_T_20170111-20170115.nc
/gpfs/fs7/dfo/hpcmc/comda/stod000/RUN_DIR/Auto-restart/CREG025_LIM3_CANOE/CREG025_LIM3_CANOE-VJC016f/CDF/CREG025_LIM3_CANOE-VJC016f_5d_ptrc_T_20170116-20170120.nc
/gpfs/fs7/dfo/hpcmc/comda/stod000/RUN_DIR/Auto-restart/CREG025_LIM3_CANOE/CREG025_LIM3_CANOE-VJC016f/CDF/CREG025_LIM3_CANOE-VJC016f_5d_ptrc_T_20170121-20170125.nc
/gpfs/fs7/dfo/hpcmc/comda/stod000/RUN_DIR/Auto-restart/CREG025_LIM3_CANOE/CREG025_LIM3_CANOE-VJC016f/CDF/CREG025_LIM3_CANOE-VJC016f_5d_ptrc_T_20170126-20170130.nc
/gpfs/fs7/dfo/hpcmc/comda/stod000/RUN_DIR/Auto-restart/CREG025_LIM3_CANOE/CREG025_LIM3_CANOE-VJC016f/CDF/CREG025_LIM3_CANOE-VJC016f_5d_ptrc_T_20170131-20170204.nc


/tmp/ipykernel_532622/3677473921.py:64: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Y0 = data.variables[variable][:]


0.0 534.2644988029234 81.52490301851286 0.0
4 4 4 28


/tmp/ipykernel_532622/3677473921.py:46: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Y0 = data.variables[variable][:]


/gpfs/fs7/dfo/hpcmc/comda/stod000/RUN_DIR/Auto-restart/CREG025_LIM3_CANOE/CREG025_LIM3_CANOE-VJC016f/CDF/CREG025_LIM3_CANOE-VJC016f_5d_ptrc_T_20170205-20170209.nc
/gpfs/fs7/dfo/hpcmc/comda/stod000/RUN_DIR/Auto-restart/CREG025_LIM3_CANOE/CREG025_LIM3_CANOE-VJC016f/CDF/CREG025_LIM3_CANOE-VJC016f_5d_ptrc_T_20170210-20170214.nc
/gpfs/fs7/dfo/hpcmc/comda/stod000/RUN_DIR/Auto-restart/CREG025_LIM3_CANOE/CREG025_LIM3_CANOE-VJC016f/CDF/CREG025_LIM3_CANOE-VJC016f_5d_ptrc_T_20170215-20170219.nc
/gpfs/fs7/dfo/hpcmc/comda/stod000/RUN_DIR/Auto-restart/CREG025_LIM3_CANOE/CREG025_LIM3_CANOE-VJC016f/CDF/CREG025_LIM3_CANOE-VJC016f_5d_ptrc_T_20170220-20170224.nc
/gpfs/fs7/dfo/hpcmc/comda/stod000/RUN_DIR/Auto-restart/CREG025_LIM3_CANOE/CREG025_LIM3_CANOE-VJC016f/CDF/CREG025_LIM3_CANOE-VJC016f_5d_ptrc_T_20170225-20170301.nc
0.0 560.3854282924107 82.02055916995964 0.0
1 6 0 31
/gpfs/fs7/dfo/hpcmc/comda/stod000/RUN_DIR/Auto-restart/CREG025_LIM3_CANOE/CREG025_LIM3_CANOE-VJC016f/CDF/CREG025_LIM3_CANOE-VJC016f_

/gpfs/fs7/dfo/hpcmc/comda/stod000/RUN_DIR/Auto-restart/CREG025_LIM3_CANOE/CREG025_LIM3_CANOE-VJC016f/CDF/CREG025_LIM3_CANOE-VJC016f_5d_ptrc_T_20171003-20171007.nc
/gpfs/fs7/dfo/hpcmc/comda/stod000/RUN_DIR/Auto-restart/CREG025_LIM3_CANOE/CREG025_LIM3_CANOE-VJC016f/CDF/CREG025_LIM3_CANOE-VJC016f_5d_ptrc_T_20171008-20171012.nc
/gpfs/fs7/dfo/hpcmc/comda/stod000/RUN_DIR/Auto-restart/CREG025_LIM3_CANOE/CREG025_LIM3_CANOE-VJC016f/CDF/CREG025_LIM3_CANOE-VJC016f_5d_ptrc_T_20171013-20171017.nc
/gpfs/fs7/dfo/hpcmc/comda/stod000/RUN_DIR/Auto-restart/CREG025_LIM3_CANOE/CREG025_LIM3_CANOE-VJC016f/CDF/CREG025_LIM3_CANOE-VJC016f_5d_ptrc_T_20171018-20171022.nc
/gpfs/fs7/dfo/hpcmc/comda/stod000/RUN_DIR/Auto-restart/CREG025_LIM3_CANOE/CREG025_LIM3_CANOE-VJC016f/CDF/CREG025_LIM3_CANOE-VJC016f_5d_ptrc_T_20171023-20171027.nc
/gpfs/fs7/dfo/hpcmc/comda/stod000/RUN_DIR/Auto-restart/CREG025_LIM3_CANOE/CREG025_LIM3_CANOE-VJC016f/CDF/CREG025_LIM3_CANOE-VJC016f_5d_ptrc_T_20171028-20171101.nc
0.0 443.1983130670363 